<a href="https://colab.research.google.com/github/nankjs/SectionProject/blob/main/ai_14_%EA%B0%95%EC%A0%95%EC%84%9D_Section1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [66]:
o_d = pd.read_csv('/content/drive/MyDrive/ColabData/CodeStates/Sec1/vgames2.csv')
o_d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   16598 non-null  int64  
 1   Name         16598 non-null  object 
 2   Platform     16598 non-null  object 
 3   Year         16327 non-null  float64
 4   Genre        16548 non-null  object 
 5   Publisher    16540 non-null  object 
 6   NA_Sales     16598 non-null  object 
 7   EU_Sales     16598 non-null  object 
 8   JP_Sales     16598 non-null  object 
 9   Other_Sales  16598 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 1.3+ MB


In [37]:
od = o_d.copy()

# 게임 이름에서 series명 분리하여 새로운 열을 만들고 3번째 열로 삽입
od.insert(2,'Sname', od.Name[od.Name.str.contains(':')].str.split(':').str[0])
od.Sname = od.Sname.fillna('FirstEd')

od = od.iloc[:,1:] # 이전 인덱스 열 삭제
od['Year'] = od['Year'].astype('Int64').astype('str') # 연도를 문자열로 변환

# 출고량 필드의 단위를 K 즉 1,000단위로 해석하고 문자열 K, M을 단순 입력 실수로 보고 삭제
# 소수로 표현된 출고량 값 전체를 의미있게 해석하는 방법이라 생각됨.
od.iloc[:,6:10] = od.iloc[:,6:10].replace(r'[a-zA-Z]',r'',regex = True)
od.iloc[:,6:10] = od.iloc[:,6:10].astype('float') * 1000
od['sumSales'] = od.iloc[:,6:10].sum(axis = 1)

# 실수형을 정수형으로 변환
for i in range(6,11,1):
  od.iloc[:,i] = od.iloc[:,i].astype(int)

# 중복행 삭제 후 재인덱스.
print(od[od.duplicated()])
od = od.drop_duplicates()
od.reset_index(drop=True)

od.info()
od.describe()

                             Name         Sname Platform  Year    Genre  \
9184  Wii de Asobu: Metroid Prime  Wii de Asobu      Wii  <NA>  Shooter   

     Publisher  NA_Sales  EU_Sales  JP_Sales  Other_Sales  sumSales  
9184  Nintendo         0         0        20            0        20  
<class 'pandas.core.frame.DataFrame'>
Int64Index: 16597 entries, 0 to 16597
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Name         16597 non-null  object
 1   Sname        16597 non-null  object
 2   Platform     16597 non-null  object
 3   Year         16597 non-null  object
 4   Genre        16547 non-null  object
 5   Publisher    16539 non-null  object
 6   NA_Sales     16597 non-null  int64 
 7   EU_Sales     16597 non-null  int64 
 8   JP_Sales     16597 non-null  int64 
 9   Other_Sales  16597 non-null  int64 
 10  sumSales     16597 non-null  int64 
dtypes: int64(5), object(6)
memory usage: 1.5+ MB


,NA_Sales,EU_Sales,JP_Sales,Other_Sales,sumSales
count,1.659700e+04,1.659700e+04,1.659700e+04,16597.000000,1.659700e+04
mean,2.530295e+03,1.080834e+03,5.364451e+02,280.405435,4.427980e+03
std,6.928059e+04,3.181653e+04,2.393439e+04,7259.117858,8.039555e+04
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,6.000000e+01
50%,8.000000e+01,2.000000e+01,0.000000e+00,10.000000,1.800000e+02
75%,2.400000e+02,1.100000e+02,4.000000e+01,40.000000,5.000000e+02
max,5.170000e+06,2.560000e+06,2.780000e+06,560000.000000,5.179180e+06


In [38]:
# 결측치 검토
print(od.isnull().sum())

nullGenre = od.Genre.isnull()
nullPubli = od.Publisher.isnull()
nullYear = od.Year == '<NA>'
nullDel = nullGenre | nullPubli | nullYear
# odm = od[nullDel]
print('결측치 수 : ', nullDel.sum())


Name            0
Sname           0
Platform        0
Year            0
Genre          50
Publisher      58
NA_Sales        0
EU_Sales        0
JP_Sales        0
Other_Sales     0
sumSales        0
dtype: int64
결측치 수 :  356


In [64]:
# Name이 중복값이 거의 없어 id역할을 할 수 있을 것으로 가정하고 채움
odm = od[od.Genre.isnull()]
odm_cmpl = od[od.Genre.notnull()]
print('장르 결측치 / 전체 레코드 수 : ' , odm.shape[0], ' / ', od.shape[0])


장르 결측치 / 전체 레코드 수 :  50  /  16597


# 새 섹션

In [ ]:
'''
출고량 필드 중 K와 M이 삽입된 값들을 1,000과 1,000,000 단위의 표시로 보았을 때 처리
그러나 소수 이하로 표시된 다른 값들을 해석할 수 없어 방향을 바꿈

t_m = od[od.iloc[:,6].str.contains('M')].iloc[:,6].str.extract('(\d+.\d+|\d+)').reset_index()
t_m.iloc[:,1] = pd.to_numeric(t_sV.iloc[:,1])*1000000
t_m.iloc[:,1].astype(str)
od.iloc[t_m.iloc[:,0],6] = t_m.iloc[:,1]

t_k = od[od.iloc[:,6].str.contains('K')].iloc[:,6].str.extract('(\d+.\d+|\d+)').reset_index()
t_k.iloc[:,1] = pd.to_numeric(t_sV.iloc[:,1])*1000
t_k.iloc[:,1].astype(str)
od.iloc[t_k.iloc[:,0],6] = t_k.iloc[:,1]

od.iloc[:,6] = pd.to_numeric(od.iloc[:,6])

'''
